# Converting AHT to CRT using Session Level Data
## Terms
$T_n = Inbound Time$

$T_o = Outbound Time$

$N_n = NumberInboundCalls$

$N_o = NumberOutboundCalls$

## Metrics
## $AHT=\frac{T_n}{N_n}$

## $CRT=\frac{T_n+T_o}{N_n}$

## Therefore,
## $\mathbb{E}[CRT] = \mathbb{E}[AHT]+\mathbb{E}[T_o]*\frac{N_o}{N_n}$

Meaning the effect on $CRT$ is the effect on $AHT$ times the ratio of inbound calls to outbound calls.

In [11]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string.replace('$DATE$','2023-08-28')

def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def run_query(query):
    print("Query Start " + timestamp())
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    print("Query End " + timestamp())
    return df
    

def add_conditionals(df):
    df['Date']=pd.to_datetime(df['Date'])
    df['TestGroup'] = (df['Employee'].isin((
        '364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275'
        ))).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    for queue in df['TaskQueue'].unique():
        df[queue] = (df['TaskQueue']==queue).astype(int)
    df['Constant'] =1
    return df

# Run Regression to Calc $\mathbb{E}[AHT]$ and $N_n$

In [12]:
df = run_query(file_to_string('AHT And Sales.sql'))
df = add_conditionals(df)

df['Having ExpertAssist Effect on Onboarding']=df['VZN_Onboarding_2628']*df['ExpertAssistOnForThisUser']
df['Having ExpertAssist Effect on MTS Bundle']=df['VZN_MTS_Bundle_EN_2233']*df['ExpertAssistOnForThisUser']
df['Having ExpertAssist Effect when Offering']= df['Offered']*df['ExpertAssistOnForThisUser']
df['Having ExpertAssist Effect when Accepted']= df['Accepted']*df['ExpertAssistOnForThisUser']
model = model = sm.OLS(
    endog=df['HandleTime'],
    exog=df[[
        # 'Having ExpertAssist Effect on Onboarding',
        # 'Having ExpertAssist Effect on MTS Bundle',
        # 'Having ExpertAssist Effect when Offering',
        # 'Having ExpertAssist Effect when Accepted',
        'Weeks Ago',
        'ExpertAssistOnForThisUser',
        'Constant','TestGroup',
        'Offered','Accepted'
        ]]).fit()
display(model.summary())
Aht_Reduction = model.params['ExpertAssistOnForThisUser']
display(Aht_Reduction)
Number_Inbound = df['Constant'].sum()

# display(df.head(),df.describe())
df['Weeks Ago']=df['Weeks Ago']*-1
px.line(pd.pivot_table(df,index='Weeks Ago',values='HandleTime',columns='TestGroup'),).show()
px.line(pd.pivot_table(df,index='Date',values='HandleTime',columns='TestGroup')).show()
# display(df.columns)
# display(pd.pivot_table(df,index=['TestGroup','AfterExpertAssistLaunch'],values=['Offered','Accepted']))

# del df, model

Query Start 2023-10-19 10:55:37 AM


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_14872\3073750119.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Query End 2023-10-19 10:55:42 AM


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             HandleTime   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     228.7
Date:                Thu, 19 Oct 2023   Prob (F-statistic):          1.68e-236
Time:                        10:55:42   Log-Likelihood:            -1.3086e+05
No. Observations:               15897   AIC:                         2.617e+05
Df Residuals:                   15891   BIC:                         2.618e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Weeks Ago                   -12.7538      4.400     -2.899      0.004     -21.377      -4.130
ExpertAssistOnForThisUser   -42.3169     25.339     -1.670      0.095     -91.983       7.350
Constant                    917.5001     21.454     42.765      0.000     875.447     959.553
TestGroup                    61.8134     18.722      3.302      0.001      25.117      98.510
Offered                     278.6803     15.177     18.362      0.000     248.932     308.429
Accepted                    689.2504     29.170     23.629      0.000     632.073     746.427
==============================================================================
Omnibus:                     7618.385   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            60428.754
Skew:                           2.154   Prob(JB):                         0.00
Kurtosis:                      11.525   Cond. No.                         18.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

-42.31686326925457

# Run Regression to Calc $\mathbb{E}[T_o]$ and $N_o$

In [13]:
df = run_query(file_to_string('OutboundTime.SQL'))
df = add_conditionals(df)

model = sm.OLS(endog=df['ResolveTime'],exog=df[['Constant','ExpertAssistOnForThisUser','TestGroup']]).fit()
display(model.summary())
OutboundReduction = model.params['ExpertAssistOnForThisUser']
display(OutboundReduction)
Number_Outbound = df['Constant'].sum()

# display(df.head(),df.describe())

del df, model

Query Start 2023-10-19 10:55:42 AM


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_14872\3073750119.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Query End 2023-10-19 10:55:46 AM


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            ResolveTime   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.192
Date:                Thu, 19 Oct 2023   Prob (F-statistic):            0.00558
Time:                        10:55:46   Log-Likelihood:                -69045.
No. Observations:                8880   AIC:                         1.381e+05
Df Residuals:                    8877   BIC:                         1.381e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Constant                    289.1504      8.470     34.137      0.000     272.546     305.754
ExpertAssistOnForThisUser   -39.5329     17.675     -2.237      0.025     -74.181      -4.885
TestGroup                    47.9146     15.032      3.187      0.001      18.448      77.382
==============================================================================
Omnibus:                     8452.585   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           552087.132
Skew:                           4.491   Prob(JB):                         0.00
Kurtosis:                      40.569   Cond. No.                         4.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

-39.53286737519106

# $\mathbb{E}[CRT] = \mathbb{E}[AHT]+\mathbb{E}[T_o]*\frac{N_o}{N_n}$

In [16]:
CRT_Reduction = Aht_Reduction+OutboundReduction*(Number_Outbound/Number_Inbound)
print(f"CRT={CRT_Reduction:.2f} = {Aht_Reduction:.2f}+{OutboundReduction:.2f}*{Number_Outbound/Number_Inbound:.2%}")
print(Number_Inbound)
print(Number_Outbound)

CRT=-64.40 = -42.32+-39.53*55.86%
15897
8880
